2024.0509 研究按照概率进行采样，设计的方法有: dataset, sampler(返回一个indices,兼容dataloader层面的batch_size和drop_last), batch_sampler(自定义返回一组) 和collate_fn

In [13]:
from torch.utils.data import Dataset as Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data.sampler import Sampler
import torch
import numpy as np

In [25]:
# alphafold mimic residue index
copy_num = 1
input_seqs = ['AABCAA']
input_sequence = input_seqs[0] * copy_num
sequence_features = {'residue_index': np.array(range(len(input_sequence)))}

idx_res = sequence_features['residue_index']
L_prev = 0
Ls = [len(input_seqs[0])]*copy_num
# Ls: number of residues in each chain
for L_i in Ls[:-1]:
    idx_res[L_prev+L_i:] += 200
    L_prev += L_i
sequence_features['residue_index'] = idx_res

In [2]:
import numpy as np
x = np.load("/home/sirius/PhD/software/alphafold/output/tmp_142/TMP_142/features.pkl",allow_pickle=True)

In [4]:
x['residue_index']

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
         44,   45,   46,   47,   48,   49,   50,   51,   52,   53,   54,
         55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
         66,   67,   68,   69,   70,   71,   72,   73,  274,  275,  276,
        277,  278,  279,  280,  281,  282,  283,  284,  285,  286,  287,
        288,  289,  290,  291,  292,  293,  294,  295,  296,  297,  298,
        299,  300,  301,  302,  303,  304,  305,  306,  307,  308,  309,
        310,  311,  312,  313,  314,  315,  316,  317,  318,  319,  320,
        321,  322,  323,  324,  325,  326,  327,  328,  329,  330,  331,
        332,  333,  334,  335,  336,  337,  338,  339,  340,  341,  342,
        343,  344,  345,  346,  347,  548,  549,  5

In [2]:
class toy_dataset(Dataset):
    def __init__(self):
        self.data = torch.randn(5, 10)
        self.labels = torch.randint(0, 2, (5,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
dataset = toy_dataset()

In [33]:
weights = [0.9,0.01,0.01,0.01,0.01]
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(dataset), replacement=False)
dataloader = DataLoader(dataset, batch_size=2, sampler=sampler,drop_last=False)
# 注意, weightedRandomSampler是一个sampler，每次返回一个indices,这样可以指定batch_size和drop_last；如果是自己写batch_sampler，则需要手动返回一个batch的indices,不支持在Dataloader层面输入batch_size

In [6]:
import torch
from torch import Tensor
class WeightedRandomSampler(Sampler[int]):
    r"""Samples elements from ``[0,..,len(weights)-1]`` with given probabilities (weights).

    Args:
        weights (sequence)   : a sequence of weights, not necessary summing up to one
        num_samples (int): number of samples to draw
        replacement (bool): if ``True``, samples are drawn with replacement.
            If not, they are drawn without replacement, which means that when a
            sample index is drawn for a row, it cannot be drawn again for that row.
        generator (Generator): Generator used in sampling.

    Example:
        >>> # xdoctest: +IGNORE_WANT("non-deterministic")
        >>> list(WeightedRandomSampler([0.1, 0.9, 0.4, 0.7, 3.0, 0.6], 5, replacement=True))
        [4, 4, 1, 4, 5]
        >>> list(WeightedRandomSampler([0.9, 0.4, 0.05, 0.2, 0.3, 0.1], 5, replacement=False))
        [0, 1, 4, 3, 2]
    """

    weights: Tensor
    num_samples: int
    replacement: bool

    def __init__(self, weights, num_samples: int,
                 replacement: bool = True, generator=None) -> None:
        if not isinstance(num_samples, int) or isinstance(num_samples, bool) or \
                num_samples <= 0:
            raise ValueError(f"num_samples should be a positive integer value, but got num_samples={num_samples}")
        if not isinstance(replacement, bool):
            raise ValueError(f"replacement should be a boolean value, but got replacement={replacement}")

        weights_tensor = torch.as_tensor(weights, dtype=torch.double)
        if len(weights_tensor.shape) != 1:
            raise ValueError("weights should be a 1d sequence but given "
                             f"weights have shape {tuple(weights_tensor.shape)}")

        self.weights = weights_tensor
        self.num_samples = num_samples
        self.replacement = replacement
        self.generator = generator

    def __iter__(self) :
        # weights 需要为全部的数据集长度, num_samples表示从数据集中按照weight的概率采样N个样本
        rand_tensor = torch.multinomial(self.weights, self.num_samples, self.replacement, generator=self.generator)
        for indices in rand_tensor:
            yield [indices]
        # yield from iter(rand_tensor.tolist())

    def __len__(self) -> int:
        return self.num_samples


In [7]:
weights = [0.9,0.01,0.01,0.01,0.01]
sampler = WeightedRandomSampler(weights, len(dataset), replacement=False) # sampler定义好之后，在for 循环的时候已经定义好一个dataloader的顺序
dataloader = DataLoader(dataset,batch_sampler=sampler)


In [10]:
for i in dataloader:
    print(i)

[tensor([[-1.0441, -2.0333,  1.3254, -1.5246,  0.6038, -0.2783, -0.3543, -0.2539,
         -0.5496, -0.4995]]), tensor([0])]
[tensor([[ 0.3582, -0.6539,  2.1860,  0.1551,  1.2448, -1.4091, -0.7608,  0.0832,
         -0.2801,  1.1191]]), tensor([0])]
[tensor([[-0.3643, -0.4998,  0.5933,  0.4850,  1.2706,  0.2043, -0.5046, -0.1878,
         -0.8916,  1.3904]]), tensor([0])]
[tensor([[ 0.8679,  0.2033,  1.8755, -0.4040,  0.3463, -0.8196, -0.9209,  0.3268,
         -1.7543, -1.4313]]), tensor([1])]
[tensor([[ 1.3100, -0.6122,  0.6776, -0.1324, -0.5451,  0.5843, -0.0231, -0.7489,
          0.4617, -0.6036]]), tensor([1])]


In [ ]:
class AccedingSequenceLengthSampler(Sampler[int]): # sampler=sampler
    def __init__(self, data: List[str]) -> None:
        self.data = data

    def __len__(self) -> int:
        return len(self.data)

    def __iter__(self) -> Iterator[int]:
        sizes = torch.tensor([len(x) for x in self.data])
        yield from torch.argsort(sizes).tolist()

class AccedingSequenceLengthBatchSampler(Sampler[List[int]]): # batch_sampler=sampler
    # 按照长度进行chunk_size的batch sample
    def __init__(self, data: List[str], batch_size: int) -> None:
        self.data = data
        self.batch_size = batch_size
    def __len__(self) -> int:
        return (len(self.data) + self.batch_size - 1) // self.batch_size
    def __iter__(self) -> Iterator[List[int]]:
        sizes = torch.tensor([len(x) for x in self.data])
        for batch in torch.chunk(torch.argsort(sizes), len(self)):
            yield batch.tolist()